## <a class="anchor" id="main">FoCS Lab</a>
### Contributors: Marco Distrutti, Santosh Anand

* [1 Normalization](#normalize)
* [2 Duration Field](#duration)
* [3 Search lenders](#search-lenders)
* [4 How many loans](#borrowers)
* [5 Overall amount](#overall-amount)
* [6 Overall percentage](#overall-percentage)
* [7 Overall by country/year](#year-overall-percentage)
* [8 Lenders overall](#lenders-overall)
* [9 Country, lent vs borrowed](#country-lent-borrowed)

In [1]:
import pandas as pd
import numpy as np
import itertools

df_loan_lenders = pd.read_csv('kiva-kaggle/loans_lenders.csv')
raws = df_loan_lenders

In [2]:
loan_lenders = df_loan_lenders
df_loan_lenders

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."
...,...,...
1387427,678999,"michael43411218, carol5987, gooddogg1, chris41..."
1387428,1207353,"rjhoward1986, jeffrey6870, trolltech4460, elys..."
1387429,1206220,"vicky7746, gooddogg1, fairspirit, craig9729960..."
1387430,1206425,"rich6705, sergiiy9766, angela7509, barbara5610..."


## 1 <a class="anchor" id="normalize">Normalization</a>
#### Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.
[⇑ index](#main)

Using **Vectorized** operations our computations are extremly faster then looping over the dataset because Vectorized operations are heavily implemented in **C** procedures. So the strategy is to create both vectors (loan_id and lender) separatly and loan_id is generated by repeating id with vectorized products.

In [3]:
%%time

#FIRST AXIS
lenders = [lender for lenders in df_loan_lenders['lenders'] for lender in lenders.split(',')]

#SECOND AXIS - using vectorized operations we boost the performance using internal opimized C procedures
loan_ids = [loan_id for loan_id in df_loan_lenders['loan_id']]
cardinality = [len(lenders.split(", ")) for lenders in df_loan_lenders['lenders']]
#create a new list with repeated ids for each loan in the same original row
flatted_ids = list(itertools.chain(*[[loan_ids[i]] * cardinality[i] for i in range(0, len(loan_ids))]))

#DATAFRAME
df_loan_lenders = pd.DataFrame({'loan_id':flatted_ids, 'lender':lenders})

Wall time: 17.2 s


In [4]:
#More then 28 milions of records
df_loan_lenders

,loan_id,lender
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


## 2 <a class="anchor" id="duration">Duration</a>
#### For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.
[⇑ index](#main)

Specifying column names and types we can avoid unnecessary data and boost performance because we can read only what we need and the interpreter won't infer data types during the scan.

In [5]:
%%time
df_loans = pd.read_csv('kiva-kaggle/loans.csv',
                       usecols=['loan_id', 'disburse_time', 'planned_expiration_time', 'country_code', 'country_name', 'loan_amount', 'num_lenders_total', 'funded_amount'],
                       dtype={'loan_id': np.int32, 'disburse_time': 'str', 'planned_expiration_time': 'str', 'country_code': 'str', 'country_name': 'str', 'loan_amount': 'float', 'num_lenders_total': np.int32, 'funded_amount': 'float'})

#it should be possible to parse datetime by specifying a lambda parser in read_csv method but
#with vectorized operations we saved more then 2 minutes for this loading.
df_loans['planned_expiration_time'] = pd.to_datetime(df_loans['planned_expiration_time'])
df_loans['disburse_time']= pd.to_datetime(df_loans['disburse_time'])
df_loans["duration"] = df_loans['planned_expiration_time'] - df_loans['disburse_time']

df_loans[['disburse_time', 'planned_expiration_time', 'duration']]

Wall time: 10.4 s


,disburse_time,planned_expiration_time,duration
0,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,53 days 19:30:06
1,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,96 days 14:25:07
2,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,37 days 13:10:05
3,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,34 days 19:10:02
4,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,57 days 22:10:02
...,...,...,...
1419602,2015-11-23 08:00:00+00:00,2016-01-02 01:00:03+00:00,39 days 17:00:03
1419603,2015-11-24 08:00:00+00:00,2016-01-02 16:40:07+00:00,39 days 08:40:07
1419604,2015-11-13 08:00:00+00:00,2016-01-03 22:20:04+00:00,51 days 14:20:04
1419605,2015-11-03 08:00:00+00:00,2016-01-05 08:50:02+00:00,63 days 00:50:02


## 3 <a class="anchor" id="search-lenders">Search lenders</a>
#### Find the lenders that have funded at least twice.
[⇑ index](#main)

Pandas aggregation methods give us the possibility to create grouped dataframe and apply aggregation functions such as the occurrences counting.

In [6]:
%%time

df_lenders_multifunder = df_loan_lenders.groupby(['lender']).count().rename(columns={"loan_id": "funds"}).sort_values(by=["funds"])
df_lenders_multifunder = df_lenders_multifunder[df_lenders_multifunder["funds"] >= 2]
df_lenders_multifunder

Wall time: 13.2 s


,funds
lender,
theresa5301,2
louis2781,2
leah1252,2
william6302,2
louis2768,2
...,...
themissionbeltco,76986
nms,100360
gmct,127089


## 4 <a class="anchor" id="borrowers">How many loans</a>
#### For each country, compute how many loans have involved that country as borrowers.
[⇑ index](#main)

There is a problem with Namibia country, in all records corresponding to this Country we haven't the **country_code**, I take this opportunity to upload country data but we noticed that even from country_stats.csv Namibia has null value in **country_code**. So we decided to force the setting of Namibia iso 2chars code to 'NA' in order to have consistent data in the original **df_loans** dataset.

In [7]:
df_loans[df_loans["country_code"].isnull()][['loan_id', 'country_code', 'country_name']]

,loan_id,country_code,country_name
82889,991853,NaN,Namibia
156970,513472,NaN,Namibia
598087,851360,NaN,Namibia
684876,1068159,NaN,Namibia
971827,998555,NaN,Namibia
1134818,1147866,NaN,Namibia
1214923,851368,NaN,Namibia
1281022,1147852,NaN,Namibia
1415763,1068167,NaN,Namibia


In [8]:
df_countries = pd.read_csv('kiva-kaggle/country_stats.csv')
df_countries[df_countries["country_code"].isnull()]

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
115,Namibia,NaN,NAM,Africa,Southern Africa,2533794,28.7,0.640007,65.062,11.657589,6.676,9769.848507,Namibia


In [9]:
#FIXING DF_LOANS
#it should be fixed applying a function to the dataset that will set a 'NA' value where a null value is found
#but we found this two step solution (divide two subsets and append them again) very faster against the apply function

#Vectorized operations are more optimized then the apply.

df_namibia = df_loans[df_loans["country_code"].isnull()]
df_namibia.country_code = ['NA'] * len(df_namibia)
df_loans = df_loans[df_loans["country_code"].notnull()].append(df_namibia)

C:\Users\marco\Anaconda3\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [10]:
%%time
df_countries_borrows = df_loans[['country_code', 'country_name', 'loan_id']].groupby(['country_name']).count().rename(columns={"loan_id": "borrows"}).sort_values(by=["borrows"])
df_countries_borrows

Wall time: 164 ms


,country_code,borrows
country_name,,
Mauritania,1,1
Papua New Guinea,1,1
Uruguay,1,1
Botswana,1,1
Canada,1,1
...,...,...
El Salvador,64037,64037
Cambodia,79701,79701
Peru,86000,86000


## 5 <a class="anchor" id="overall-amount">Overall amount</a>
#### For each country, compute the overall amount of money borrowed
[⇑ index](#main)

In [11]:
%%time
df_countries_borrowed = df_loans[['country_code', 'country_name', 'loan_amount']].groupby(['country_code', 'country_name'], as_index=False).sum().rename(columns={"loan_amount": "borrowed"}).sort_values(by=["borrowed"])
df_countries_borrowed

Wall time: 200 ms


,country_code,country_name,borrowed
30,GZ,Gaza,5000.0
85,UY,Uruguay,8000.0
12,BW,Botswana,8000.0
89,VU,Vanuatu,9250.0
87,VI,Virgin Islands,10000.0
...,...,...,...
40,KH,Cambodia,51613525.0
66,PY,Paraguay,53964700.0
38,KE,Kenya,66735975.0
60,PE,Peru,79437775.0


## 6 <a class="anchor" id="overall-amount">Overall percentage</a>
#### Like the previous point, but expressed as a percentage of the overall amount lent.
[⇑ index](#main)

In [12]:
total_borrowed = df_countries_borrowed.borrowed.sum()
#vectorized operation
total_borrowed_perc = (df_countries_borrowed.borrowed / total_borrowed) * 100

df_countries_borrowed_perc = pd.DataFrame({'country_name': df_countries_borrowed.country_name, 'borrowed':total_borrowed_perc})
df_countries_borrowed_perc

,country_name,borrowed
30,Gaza,0.000423
85,Uruguay,0.000677
12,Botswana,0.000677
89,Vanuatu,0.000783
87,Virgin Islands,0.000846
...,...,...
40,Cambodia,4.368706
66,Paraguay,4.567716
38,Kenya,5.648711
60,Peru,6.723825


## 7 <a class="anchor" id="year-overall-percentage">Overall by country/year</a>
#### Like the three previous points, but split for each year (with respect to disburse time).
[⇑ index](#main)

The following dataset is the grouped amount by **country_name** and **year**

In [13]:
df_loans_year = df_loans[['country_name', 'disburse_time', 'loan_amount']].rename(columns={"loan_amount": "borrowed"})
df_loans_year['year'] = df_loans.disburse_time.dt.year

df_countries_borrowed_year = df_loans_year.groupby(['country_name', 'year']).sum().sort_values(by=["borrowed"])
df_countries_borrowed_year

borrowed
country_name year              
Paraguay     2018.0        50.0
Pakistan     2018.0       150.0
Philippines  2018.0       300.0
Mexico       2018.0       475.0
Thailand     2012.0      1050.0
...                         ...
Kenya        2015.0  10257950.0
Philippines  2014.0  13961450.0
             2015.0  16083375.0
             2016.0  16218925.0
             2017.0  17096900.0

[748 rows x 1 columns]

We need the total amounts for each year

In [14]:
overall_years = df_loans_year.groupby(['year']).sum()
overall_years

,borrowed
year,
2005.0,102850.0
2006.0,1376575.0
2007.0,15446525.0
2008.0,39423050.0
2009.0,59689475.0
2010.0,72609150.0
2011.0,93699300.0
2012.0,119977575.0
2013.0,132043925.0


Again, with vectorized operations we can boost our computation. In this case a division by matching the indexes will be done.
In case of an uncomparable matching is given the following error will be displayed: **ValueError: cannot join with no overlapping index names**

In [15]:
df_countries_borrowed_year = df_countries_borrowed_year / overall_years * 100
df_countries_borrowed_year

borrowed
country_name year             
Paraguay     2018.0   0.005050
Pakistan     2018.0   0.015151
Philippines  2018.0   0.030302
Mexico       2018.0   0.047979
Thailand     2012.0   0.000875
...                        ...
Kenya        2015.0   6.582544
Philippines  2014.0   9.168852
             2015.0  10.320729
             2016.0  10.075220
             2017.0  10.296010

[748 rows x 1 columns]

## 8 <a class="anchor" id="lenders-overall">Lenders overall</a>
#### For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.
[⇑ index](#main)

The total number of lenders, for each loan, is specified by the **num_lenders_total** feature, and the effective funded amount by the **funded_amount** feature. As you can see from the next table the loan_amount can be different from the funded amount.

In [16]:
df_loans[df_loans["loan_amount"] != df_loans["funded_amount"]][['loan_id', 'loan_amount', 'funded_amount']]

,loan_id,loan_amount,funded_amount
22,957461,225.0,100.0
25,959145,300.0,225.0
36,416929,1000.0,425.0
46,419381,875.0,275.0
50,421569,1200.0,325.0
...,...,...,...
1419525,1089430,225.0,125.0
1419538,1235452,3000.0,600.0
1419545,987838,1025.0,725.0
1419564,1151269,8750.0,50.0


So now we can merge all the informations and compute the desired values, in this step **funded_amount** will change its original representation and will be reshaped with a **single quota** funded by lenders. A vectorized operation will be applied for this scope.

In [17]:
df_loan_funds = df_loans[['loan_id', 'funded_amount', 'num_lenders_total']]
df_loan_funds.funded_amount = df_loan_funds.funded_amount / df_loan_funds.num_lenders_total
df_loan_funds = df_loan_funds[['loan_id', 'funded_amount']].set_index('loan_id')
df_loan_funds

,funded_amount
loan_id,
657307,41.666667
657259,36.363636
658010,25.000000
659347,29.761905
656933,28.333333
...,...
998555,26.388889
1147866,27.322404
851368,26.100629


In [18]:
df_lender_funds = pd.merge(df_loan_lenders, df_loan_funds, on="loan_id").drop(columns=["loan_id"]).groupby(['lender']).sum()
df_lender_funds

,funded_amount
lender,
000,1485.309656
00000,1249.127691
0002,2201.180463
00mike00,38.461538
0101craign0101,2424.088932
...,...
zzanita,62.500000
zzcyna7269,44.285714
zzinnia,32.758621


## 9 <a class="anchor" id="country-lent-borrowed">Country, lent vs borrowed</a>
#### For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.
[⇑ index](#main)

In [19]:
df_lenders = pd.read_csv('kiva-kaggle/lenders.csv')
df_lenders

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
2349169,janet7309,Janet,NaN,NaN,NaN,1342097163,NaN,NaN,NaN,NaN,0
2349170,pj4198,NaN,NaN,NaN,NaN,1342097515,NaN,NaN,NaN,NaN,0
2349171,maria2141,Maria,NaN,NaN,US,1342099723,NaN,NaN,2.0,NaN,0
2349172,simone9846,Simone,NaN,NaN,NaN,1342100213,NaN,NaN,NaN,NaN,0


In [20]:
df_lenders.count()

permanent_name       2349174
display_name         2346406
city                  729868
state                 635693
country_code          890539
member_since         2349174
occupation            504660
loan_because          174322
loan_purchase_num    1454893
invited_by            496825
num_invited          2349174
dtype: int64

The tables above highlights how many **missing values** we have in country_code and that the permanent_name seems to be the primary key, given the difference between the **permanent_name** key and the **country_code** occurrences we have more then 50% of missing values. Now the most important thing is to fill those missing values with an equal distribution in respect of known countries.

In [21]:
#initialize a subset of original dataset
length = len(df_lenders)
df_lenders_sub = df_lenders[['permanent_name', 'country_code', 'num_invited']].rename(columns={"num_invited": "total"})
df_lenders_sub.total = [0] * length
df_lenders_sub

#take all the lenders with missing country values
df_lenders_no_country = df_lenders_sub[df_lenders_sub['country_code'].isnull()]
length_no_country = len(df_lenders_no_country)

#take all the lenders with missing country values
df_lenders_with_country = df_lenders_sub[df_lenders_sub['country_code'].notnull()]

#distribution of known countries
df_country_lenders = df_lenders_sub[['country_code', 'total']].groupby(['country_code']).count()
length_country = df_country_lenders['total'].sum()
df_country_lenders["relative"] = df_country_lenders["total"] / length_country
df_country_lenders["to_fill"] = df_country_lenders["relative"] * length_no_country
df_country_lenders

,total,relative,to_fill
country_code,,,
AD,15,0.000017,24.568857
AE,1043,0.001171,1708.354497
AF,228,0.000256,373.446621
AG,8,0.000009,13.103390
AI,4,0.000004,6.551695
...,...,...,...
YE,195,0.000219,319.395136
YT,2,0.000002,3.275848
ZA,1051,0.001180,1721.457887


Above table represent the country known distribution with absolute **total** values, **relative** values in respect of total known country distribution and, finally, the simulation **to_fill** expressed in real values in order to preserve the same proportionality.

**Following assertions will check the cardinality and proportion of our computations**

In [22]:
assert(abs(df_country_lenders.to_fill.sum() - length_no_country) < 0.0001), "Cardinality error: the total number of countries to be entered differs from the total number of missing countries"
assert(abs(df_country_lenders.to_fill / length_no_country - df_country_lenders.relative < 0.0001).all()), "Proportionality error: proportion between countries to be entered in respect to the total missing values differs from the relative known distribution"

We have to apply an approximation in final total values to be entered because we cannot insert 24.56 AD values but 24 or 25. So using the integer round approximation we will continue our work.

In [23]:
df_country_lenders["to_fill_final"] = df_country_lenders.to_fill.round()
delta = int(df_country_lenders.to_fill_final.sum() - df_country_lenders.to_fill.sum()) #there will be a difference, such as -7
single = delta / abs(delta) #we want 1 or -1
delta_vector = ([single] * abs(delta)) + [0] * (len(df_country_lenders) - abs(delta)) #create the following (1,...,1,0,...,0)
df_country_lenders.to_fill_final = df_country_lenders.to_fill_final - delta_vector #adapt final values

In [24]:
assert(df_country_lenders.to_fill_final.sum() == length_no_country), "Cardinality error: the total number of countries to be entered differs from the total number of missing countries"

In [25]:
missing_countries = [[df_country_lenders.iloc[i].name] * int(df_country_lenders.iloc[i]["to_fill_final"]) for i in range(len(df_country_lenders))]
missing_countries = list(itertools.chain(*missing_countries)) #flat list
assert(len(missing_countries) == length_no_country), "Cardinality error: the total number of countries to be entered differs from the total number of missing countries"

No more **missing_values** in the next dataset. We need the total funded by lenders so we will add it to the dataset

In [26]:
df_lenders_no_country.country_code = missing_countries
df_lenders_nomissing = df_lenders_with_country.append(df_lenders_no_country)[["permanent_name", "country_code"]].rename(columns={'permanent_name': 'lender'})
df_lenders_nomissing = df_lenders_nomissing.merge(df_lender_funds, on='lender')
df_countries_funded = df_lenders_nomissing[['country_code', 'funded_amount']].groupby(['country_code']).sum()

The overall amount of money borrowed for each country was computed during the fifth point and we have these values in **df_countries_borrowed** dataframe. Note that countries do not necessarly appear on both lists. We apply a country_code

In [27]:
print("Countries for money borrow: ", len(df_countries_borrowed))
print("Countries for money funded: ", len(df_countries_funded))

Countries for money borrow:  96
Countries for money funded:  216


We can manage this situation with a full outer join because we want all the countries but we cannot know a priori if a country is in one list or in the other. After this, we cannot apply directly the difference, we have to convert null values to 0. We tested two type of strategies. **loc** strategy for updating the values **in loco** and **numpy.where**

From the following benchmark it is clear the reason why we used **np.where**.

```python
%timeit df_country_diff.loc[df_country_diff['borrowed'].isnull(), 'borrowed'] = 0
#1.05 ms ± 7.49 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
%timeit df_country_diff['borrowed'] = np.where(df_country_diff['borrowed'].isnull(), 0, df_country_diff['borrowed'])
#221 µs ± 12.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
```

In [28]:
df_country_diff = pd.merge(df_countries_borrowed[['country_code', 'borrowed']].set_index('country_code'),df_countries_funded,on='country_code', how='outer')

df_country_diff.borrowed = np.where(df_country_diff.borrowed.isnull(), 0, df_country_diff.borrowed)
df_country_diff.funded_amount = np.where(df_country_diff.funded_amount.isnull(), 0, df_country_diff.funded_amount)
df_country_diff["difference"] = df_country_diff.funded_amount - df_country_diff.borrowed
df_country_diff

,borrowed,funded_amount,difference
country_code,,,
GZ,5000.0,27.083333,-4972.916667
UY,8000.0,13014.441020,5014.441020
BW,8000.0,178.928571,-7821.071429
VU,9250.0,15098.241997,5848.241997
VI,10000.0,1649.611330,-8350.388670
...,...,...,...
UM,0.0,804.918555,804.918555
UZ,0.0,721.966631,721.966631
VA,0.0,93.636364,93.636364
